<a href="https://colab.research.google.com/github/lujain618/Deepfake_Detection_With_XAI/blob/main/HybridModel(CNN%2BLSTM).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import TimeDistributed, LSTM, Dense, Dropout, GlobalAveragePooling2D
import numpy as np
import os

# Define CNN (ResNet) feature extractor
base_cnn = ResNet50(weights='imagenet', include_top=False, input_shape=(112, 112, 3))
base_cnn = Model(inputs=base_cnn.input, outputs=GlobalAveragePooling2D()(base_cnn.output))

# Define full model
model = Sequential([
    TimeDistributed(base_cnn, input_shape=(30, 112, 112, 3)),  # 30 frames per video
    LSTM(256, return_sequences=False),  # Capture temporal dependencies
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification (real/fake)
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Summary
model.summary()

# Data generator function to handle large datasets efficiently
def data_generator(dataset_path, batch_size=16):
    while True:
        X, y = [], []
        for label in ['0', '1']:
            label_path = os.path.join(dataset_path, label)
            for video_folder in os.listdir(label_path):
                video_path = os.path.join(label_path, video_folder)
                frames = []
                for frame_file in sorted(os.listdir(video_path)):
                    frame_path = os.path.join(video_path, frame_file)
                    try:
                        frame = np.load(frame_path)
                        if frame.size == 0:
                            continue
                        frames.append(frame)
                    except:
                        continue
                if len(frames) == 30:
                    X.append(np.array(frames))
                    y.append(int(label))

                if len(X) == batch_size:  # Yield a batch when it's full
                    yield np.array(X), np.array(y)
                    X, y = [], []

# Define dataset loader
DATASET_PATH = '/content/drive/MyDrive/DeepfakeDataset/train_processed/'
batch_size = 16
train_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(DATASET_PATH, batch_size),
    output_signature=(
        tf.TensorSpec(shape=(None, 30, 112, 112, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None,), dtype=tf.int32)
    )
)

# Train using the data generator
model.fit(train_dataset, epochs=20, steps_per_epoch=100, callbacks=[
    tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)
])


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ time_distributed (TimeDistributed)   │ (None, 30, 2048)            │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 256)                 │       2,360,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 25,981,057 (99.11 MB)

 Trainable params: 25,927,937 (98.91 MB)

 Non-trainable params: 53,120 (207.50 KB)

Epoch 1/20


KeyboardInterrupt: 